## FACE RECOGNITION ATTENDANCE MANAGEMENT SYSTEM

# Generating dataset

In [4]:
import cv2

Use can use cv2 by installing pip install opencv-python

We will use CascadeClassifier

In [2]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
    
    # To separate face from image
    def face_cropped(img):
        # Converting image to greyscale
        gray =cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 1.1 is the scaling factor
        # 4 minimum diverse
        faces = face_classifier.detectMultiScale(gray, 1.1, 4)

        # () denotes empty
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cv2.rectangle(img, (x,y), (x+w, y+w), (0,0,255), 2)
            cropped_face = img[y:y+h,x:x+w]

        return cropped_face
    
    # Opening web cam
    # 0 for internal camera
    # 1 for external camera
    cap = cv2.VideoCapture(0)
    img_id = 0

    while True:
        # this is our image frame
        ret, frame = cap.read()
        # if face is found
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            # save image in this data folder
            file_name_path = "data/" + "Swapnita_" + str(img_id) + ".jpg"
            # to save image face in file_name_path
            cv2.imwrite(file_name_path, face)

            # putText method to count
            # to put image id on the face
            # (50,50) is the origin point
            # font in which text is to be written
            # font scale 1
            # 0,255,0 is green in RGB
            # 2 is thickness
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

            # Cropped Face is title and face is displayed
            cv2.imshow("Cropped_Face", face)
            # if we press enter or image id is 1000 we break the loop
            # 13 is the ASCII value of enter
            if cv2.waitKey(1)==13 or int(img_id)==50:
                break

    # releasing the camera and distroying all windows
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed!")

# Calling the created function 
generate_dataset()

<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\swapn\AppData\Local\Temp\ipykernel_33332\1210666039.py:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


NameError: name 'cv2' is not defined

# Create Label

In [3]:
import numpy as np

we can install numpy by using pip install numpy

In [4]:
# image_name is the parameter of the created function
def my_label(image_name):
    name = image_name.split('.')[-3]
    # if you have 2 people in your dataset
    if name == "Swapnita":
        return np.array([1,0])
    elif name == "Shreya":
        return np.array([0,1])
    
    # If you have 3 people in the dataset
    #if name == "Swapnita":
    #    return np.array([1,0,0])
    #elif name == "Shruti":
    #    return np.array([0,1,0])
    #elif name == "Khushi":
    #    return np.array([0,0,1])   

# Create data
Now, we need to create dataset from the collected images

In [6]:
import os
from random import shuffle
from tqdm import tqdm

In [ ]:
def my_data():
    # Creating an empty list tqdm
    data = []
    for img in tqdm(os.listdir("data")):
        path = os.path.join("data",img)
        img_data = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        data.append([np.array(img_data)])
    shuffle(data)
    return data

In [ ]:
data = my_data()

In [ ]:
train = data[:2400]
test = data[2400:]
X_train = np.array([i[0] for i in train]).reshape(-1, 50, 50, 1)
print(X_train.shape)
Y_train = [i[1] for i in train]
X_test = np.array([i[0] for i in test]).reshape(-1, 50, 50, 1)
print(X_test.shape)
Y_test = [i[1] for i in test]

# Creating the model

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
fromtflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [ ]:
tf.reset_default_graph()
convnet = input_data(shape=[50,50,1])
convnet = conv_2d(convnet, 32, 5, activation='relu')
#32 filters and stride=5 so that the filter will move 5 pixel or unit at a time
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')

convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')

convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')

convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')

convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 3, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate = 0.001, loss='categorical_crossenthropy')
model = tflearn.DNN(convnet, tensorboard_verbose=1)
model.fit(X.train, Y_train, n_epoch=12, validation_set=(X_test, Y_test), show_metric = True, run_id="FRS")

# Visualizing the data and making prediction

In [ ]:
def data_for_visualization():
    Vdata = []
    for img in tqdm(os.listdir("Images for visualization")):
        path = os.path.join("Images for visualization", img)
        img_num = img.split('.')[0]
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50, 50))
        Vdata.append([np.array(img_data), img_num])
    shuffle(Vdata)
    return Vdata

In [ ]:
Vdata = data_for_visualization()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(20,20))
for num, data in enumerate(Vdata[:20]):
    img_data = data[0]
    y = fig.add_subplot(5,5, num+1)
    image = img_data
    data = img_data.reshape(50, 50, 1)
    model_out = model.predict([data])[0]

    if np.argmax(model_out) == 0:
        my_label = 'Swapnita'
    elif np.argmax(model_out) == 1:
        my_label = 'Shruti'
    else:
        my_label = 'Khushi'

    y.imshow(image, cmap='gray')
    plt.title(my_label)

    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()